# Process Lane Detection Log

From all the stats in the "metadata.csv" produced by lane detection:

* For each point, find the corresponding "intersection" nodes in OSM for the way segment
    * Find the nearest way, excluding unnamed (roundabouts etc. that are really intersections themselves)
    * On that way, find the three nearest points
    * The nearest point is one intersection node
    * If the point is further from the second-nearest point than the third-nearest point, then use the third-nearest
    
    |-------*-|-|
    
* If the distance from the previous point is less than a threshold number of metres, then disregard the point as a duplicate that might otherwise skew our assessment.  E.g. car waiting at intersection for many frames.

* What is the difference between the slopes of the potential bike lane/shoulder?

* What is the difference between the intercepts at the top line?

==> Set thresholds for these

==> Count what proportion of points in a way section meet both criteria

==> Compare to ground truth for a suburb to select a score/proportion to use for a yes/no answer

==> Draw on a map

In [1]:
import_directory = 'dashcam_tour_mount_eliza'
#import_directory = 'dashcam_tour_frankston'

locality = 'Mount Eliza'
#locality = 'Frankston'
margin   = 20

In [2]:
import os
import sys

import pandas as pd

from shapely.geometry import Point

module_path_root = os.path.abspath(os.pardir)
if module_path_root not in sys.path:
    sys.path.append(module_path_root)
    
# Import local modules
import osm_gsv_utils.osm_walker as osm_walker

In [3]:
# Derive paths
metadata_csv = os.path.join(module_path_root, 'data_sources', import_directory, 'split', 'metadata.csv')

locality_clean = locality.replace(' ', '_')

filename_main       = os.path.join(os.pardir, 'data_sources', 'Locality_' + locality_clean + '.osm')
filename_margin     = os.path.join(os.pardir, 'data_sources', 'Locality_' + locality_clean + '_margin.osm')
locality_margin     = '{0:s}_{1:d}m'.format(locality_clean, margin)

In [4]:
# Load OSM data
walker = osm_walker(filename_main, filename_margin, verbose=False)

In [5]:
df = pd.read_csv(metadata_csv)
df.head(5)

,filename,prefix,frame_num,lat,lon,altitude,heading,pixels_bottom,pixels_top,left_slope2,left_int2,left_slope1,left_int1,right_slope1,right_int1
0,E:\TensorFlow2\TFODCourse\minor_thesis\data_so...,FILE210924-100801F,0,-38.201282,145.103536,130.5,233.32,280.51,82.91,-0.3569746389425754,717.5784568371525,-0.6586782047529849,967.9134029201746,0.6712406249123887,-203.93659294864995
1,E:\TensorFlow2\TFODCourse\minor_thesis\data_so...,FILE210924-100801F,12,-38.201290,145.103523,130.5,233.32,226.36,68.27,-0.3706141135917843,731.7965031570301,-0.5981974878278502,923.574105074722,0.6377436067570221,-166.39047770345684
2,E:\TensorFlow2\TFODCourse\minor_thesis\data_so...,FILE210924-100801F,24,-38.201297,145.103511,130.5,233.32,0.00,0.00,None,None,-0.5189460444211357,852.0394576339941,0.6725104742474769,-199.95449792776338
3,E:\TensorFlow2\TFODCourse\minor_thesis\data_so...,FILE210924-100801F,36,-38.201305,145.103498,130.5,233.32,0.00,0.00,None,None,-0.5093364599617861,855.623805801633,0.6981329104758701,-232.14896477118634
4,E:\TensorFlow2\TFODCourse\minor_thesis\data_so...,FILE210924-100801F,48,-38.201312,145.103486,130.5,233.32,0.00,0.00,None,None,-0.5279331389079418,872.3841765514883,0.6105692531789348,-134.32055093255286


In [6]:
# Example point, let's just do one at a time first
p = Point(-38.201282, 145.103536)

In [7]:
way_id_start = walker.find_nearest_way_segment(p, verbose=False)
print(way_id_start)
print(walker.way_names_by_id[way_id_start])

172385200
BELLBIRD ROAD


In [13]:
way_id_start, closest_node_id1, closest_node_id2, closest_distance1, closest_distance2 = walker.find_nearest_node_pair(p, verbose=False)
print('Way: {0:s}, Intersection 1: {1:s} {2:.0f}m, Intersection 2: {3:s} {4:.0f}m'.format(
    way_id_start, closest_node_id1, closest_distance1, closest_node_id2, closest_distance2))
print(walker.node_coords[closest_node_id1])
print(walker.node_coords[closest_node_id2])

Way: 172385200, Intersection 1: 1853686745 36m, Intersection 2: 458414690 97m
[-38.2014245, 145.1031611]
[-38.2007143, 145.104377]
